# Libbitcoin BX: Transactions with relative locktime.
In this BX tutorial, we will send funds to a `checksequenceverify` encumbered output, and spend this after the locktime delay (which begins when the uxto is included in a block) has expired.
<img src="images/csv_overview.jpg" alt="drawing" style="" width="700px"/>

<hr style="border: 0.5px dashed #000;">

## 1. Restore wallet and generate spending key pairs.


In [1]:
# my_mnemonic ="word0 word1 word2 ..."
# hd_master_private=$( )
# hd_master_public=$( )

my_mnemonic="seven mail crash you unit small assume express wedding cloud work potato"
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)


### Restore your spending keys.

* `m/44'/1'/1'/0/5`

In [3]:
# hd_m_44h_1h_1h_0_5=

# privatekey_44h_1h_1h_0_5=
# publickey_44h_1h_1h_0_5=
# publickeyhash_44h_1h_1h_0_5=
# address_44h_1h_1h_0_5=

hd_m_44h_1h_1h_0_5=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0 \
| bx hd-private --index 5)

privatekey_44h_1h_1h_0_5=$(bx hd-to-ec $hd_m_44h_1h_1h_0_5)
publickey_44h_1h_1h_0_5=$(bx hd-to-public $hd_m_44h_1h_1h_0_5 | bx hd-to-ec)
publickeyhash_44h_1h_1h_0_5=$(bx sha256 $publickey_44h_1h_1h_0_5 | bx ripemd160)
address_44h_1h_1h_0_5=$(bx hd-to-public $hd_m_44h_1h_1h_0_5 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_1h_0_5
echo $publickeyhash_44h_1h_1h_0_5


mqpVyrj3iRPe7sNKht9vvnbhnDCAPdiJYv
7102c7914f7ac4e5a8055c5954b3462e0ddb419f


### Generate destination keys to send funds to.

* `m/44'/1'/2'/0/5`

In [4]:
# hd_m_44h_1h_2h_0_5=

# privatekey_44h_1h_2h_0_5=
# publickey_44h_1h_2h_0_5=
# publickeyhash_44h_1h_2h_0_5=
# address_44h_1h_2h_0_5=

hd_m_44h_1h_2h_0_5=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 2  \
| bx hd-private --index 0 \
| bx hd-private --index 5)

privatekey_44h_1h_2h_0_5=$(bx hd-to-ec $hd_m_44h_1h_2h_0_5)
publickey_44h_1h_2h_0_5=$(bx hd-to-public $hd_m_44h_1h_2h_0_5 | bx hd-to-ec)
publickeyhash_44h_1h_2h_0_5=$(bx sha256 $publickey_44h_1h_2h_0_5 | bx ripemd160)
address_44h_1h_2h_0_5=$(bx hd-to-public $hd_m_44h_1h_2h_0_5 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_2h_0_5
echo $publickeyhash_44h_1h_2h_0_5


msVCynJ5Hr5UoXE3UKMFQEhHHCajQraSK9
834cef9529c1c72fb211cf7c202d521824c9b56b


### Fetch previous UTXO's you are spending.

In [5]:
# bx fetch-tx [transaction_hash] -f json | jq ".transaction.outputs[output_index]"

bx fetch-tx 11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea -f json | jq ".transaction.outputs[5]"


{
  "address_hash": "7102c7914f7ac4e5a8055c5954b3462e0ddb419f",
  "script": "dup hash160 [7102c7914f7ac4e5a8055c5954b3462e0ddb419f] equalverify checksig",
  "value": "499550"
}


In [6]:
# previous_txid=
# previous_output_index=
# previous_output_amount=

previous_txid=11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea
previous_output_index=5
previous_output_amount=499550

<hr style="border: 0.5px dashed #000;">

## 2. Spending to a CSV encumbered output<br>
<img src="images/spend_to_csv.jpg" alt="drawing" style="" width="700px"/>

### A. Build transaction with CSV encumbered output for signing

#### Determine delay in blocks
* Encode delay value in hex (little endian)
* `CHECKSEQUENCEVERIFY` verifies a 3-byte `[delay]` value on the stack against the spending transaction<br>
* For a `[delay]` in blocks:
    * Bit flag (1 << 22) is not set.
    * First 16 bits interpreted as blocks.
    * Little endian encoded.  

In [21]:
# A five block delay: 0x000005

# delay=

delay=050000


**Note:** Don't forget to flip byte-order (3-Bytes) above for output script below.

#### Encode CSV-encumbered P2PKH output script
* **`[delay]`**
* **`CHECKSEQUENCEVERIFY`**
* `DROP`
* `DUP`
* `HASH160`
* `[public key hash]`
* `EQUALVERIFY`
* `CHECKSIG`

In [22]:
# output_script=

output_script=$(bx script-encode "[$delay] CHECKSEQUENCEVERIFY DROP DUP HASH160 [$publickeyhash_44h_1h_2h_0_5] EQUALVERIFY CHECKSIG")
echo $output_script


03050000b27576a914834cef9529c1c72fb211cf7c202d521824c9b56b88ac


#### Calculate output amount

* `400 Satoshi/sigops + 1 Satoshi/Byte` 
* Output amount(s) are previous output amount(s) minus fees

In [23]:
# tx bytes: 4 + 1 + 1*(32+4+1+1+72+1+33+4) + 1 + 1*(8+1+(6+25)) + 4 

# tx_byte_count=

tx_byte_count=$(expr 4 + 1 + 1 \* 148 + 1 + 1 \* 40 + 4)
echo $tx_byte_count


198


In [24]:
# Output amount total.

# output_amount=

output_amount=$(expr $previous_output_amount - 1 \* 400 - $tx_byte_count \* 1)
echo $output_amount


498952


#### Encode transaction for signing
* `bx tx-encode -i [prevout txid:index:sequence] -o [output script:amount]`


In [25]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

# csv_tx=

csv_tx=$(bx tx-encode \
--input $previous_txid:$previous_output_index:4294967295 \
--output $output_script:$output_amount)
echo $csv_tx



0100000001ea03ec3bcb3417c58ab4d61677d26498f974fd979734e3697ffc77507f58a8110500000000ffffffff01089d0700000000001f03050000b27576a914834cef9529c1c72fb211cf7c202d521824c9b56b88ac00000000


### Sign the transaction.


**Fetch previous output script**

In [26]:
# bx fetch-tx --format json [previous tx id] 

# previous_output_script=

previous_output_script=$(bx fetch-tx --format json $previous_txid \
| jq -r ".transaction.outputs[5].script")
echo $previous_output_script


dup hash160 [7102c7914f7ac4e5a8055c5954b3462e0ddb419f] equalverify checksig


**Sign transaction with previous output script.**

In [27]:
# bx input-sign [private key] "previous output script" [transaction template]

# signature= 

signature=$(bx input-sign --sign_type all --index 0 $privatekey_44h_1h_1h_0_5 "$previous_output_script" $csv_tx)


**Set the input script into the finalised transaction.**

In [28]:
# bx input-set "[signature] [public key point]" [transaction template]

# csv_tx=

csv_tx=$(bx input-set --index 0 "[$signature] [$publickey_44h_1h_1h_0_5]" $csv_tx)


In [29]:
# Decode.
bx tx-decode -f json $csv_tx | jq 


{
  "transaction": {
    "hash": "0f579b753f486c10bb26547cced3a95abecb93e6c81e4ddbaf273046aa2b2be4",
    "inputs": [
      {
        "address_hash": "7102c7914f7ac4e5a8055c5954b3462e0ddb419f",
        "previous_output": {
          "hash": "11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea",
          "index": "5"
        },
        "script": "[3045022100c2e4a8513307c838d6c09f68464d2e37b05168815d2804e6f72952dfcb9519fa02205d5d4d30274dc8db192f8de79cdfc83e0541e49fff7f4c4fb8628c3a3729ad2301] [025d6ea3f5af51796062b6af01470e7b764f83e557c2212191fc55b9896505ae84]",
        "sequence": "4294967295"
      }
    ],
    "lock_time": "0",
    "outputs": [
      {
        "script": "[050000] checksequenceverify drop dup hash160 [834cef9529c1c72fb211cf7c202d521824c9b56b] equalverify checksig",
        "value": "498952"
      }
    ],
    "version": "1"
  }
}


In [30]:
# Validate.
bx validate-tx $csv_tx



The transaction is valid.


### 2.3) Validate & Broadcast the endorsed transaction
* `bx validate-tx` `[encoded transaction]`
* `bx tx-decode -f json` `[encoded transaction]` | `jq ".transaction.hash"`

In [ ]:
bx validate-tx \
01000000019dc2f9c31dacce0cfec97e0b8478726a2ac3b3585e0fbb4e9331c69a4c0b476e000000006b483045022100e9d38f238e58b3eb4642a618e0478bde90e33e5f17749336c97d8057f1b0a3d6022070e7c2f2b9cf4f0d771889dd6488d399889408ff4985ea2def6ede808de69b760121035d2725b8a5b78f8e235a92e3acc2e426ac70889785dba7020189e88caa8a45b6ffffffff012ea1bf07000000001f03040000b27576a914903d20a04087a7a843f1f6fe0dfc8ae7358e058d88ac00000000


In [ ]:
bx tx-decode -f json \
01000000019dc2f9c31dacce0cfec97e0b8478726a2ac3b3585e0fbb4e9331c69a4c0b476e000000006b483045022100e9d38f238e58b3eb4642a618e0478bde90e33e5f17749336c97d8057f1b0a3d6022070e7c2f2b9cf4f0d771889dd6488d399889408ff4985ea2def6ede808de69b760121035d2725b8a5b78f8e235a92e3acc2e426ac70889785dba7020189e88caa8a45b6ffffffff012ea1bf07000000001f03040000b27576a914903d20a04087a7a843f1f6fe0dfc8ae7358e058d88ac00000000 \
|jq ".transaction.hash"


In [ ]:
bx send-tx 01000000019dc2f9c31dacce0cfec97e0b8478726a2ac3b3585e0fbb4e9331c69a4c0b476e000000006b483045022100e9d38f238e58b3eb4642a618e0478bde90e33e5f17749336c97d8057f1b0a3d6022070e7c2f2b9cf4f0d771889dd6488d399889408ff4985ea2def6ede808de69b760121035d2725b8a5b78f8e235a92e3acc2e426ac70889785dba7020189e88caa8a45b6ffffffff012ea1bf07000000001f03040000b27576a914903d20a04087a7a843f1f6fe0dfc8ae7358e058d88ac00000000 


<hr style="border: 0.5px dashed #000;">

## 3. Spend the CSV encumbered UXTO

<img src="images/spend_from_csv.jpg" alt="drawing" style="" width="700px"/>

### A. Build p2pkh output.

#### Encode p2pkh output script

* Please send testnet coins to the following address: `n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw`

In [33]:
bx address-decode -f json n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw | jq ".wrapper.payload"


"e48199d47742b245464b1366d95ef26aa4c8bb2c"


In [34]:
# output_script=

output_script=$(bx script-encode "DUP HASH160 [e48199d47742b245464b1366d95ef26aa4c8bb2c] EQUALVERIFY CHECKSIG")
echo $output_script

76a914e48199d47742b245464b1366d95ef26aa4c8bb2c88ac


**Encode transaction with delay & set input sequence**<br>
* Transaction sequence delay must be `>=` delay in CSV output being spent.
* Sequence delay encoding in blocks:
    * Transaction version must be >= 2.
    * Bit flag (1 << 22) must be deactivated.
    * Encode block delay in first 16 bits.

In [35]:
# tx bytes: 4 + 1 + 1*(32+4+1+1+72+1+33+4) + 1 + 1*(8+1+25) + 4 

# tx_byte_count=

tx_byte_count=$(expr 4 + 1 + 1 \* 148 + 1 + 1 \* 34 + 4)
echo $tx_byte_count


192


In [36]:
# Output amount total.

# output_amount=

output_amount=$(expr $previous_output_amount - 1 \* 400 - $tx_byte_count \* 1)
echo $output_amount


498958


In [38]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

# my_tx=

my_tx=$(bx tx-encode \
--input $previous_txid:$previous_output_index:5 \
--output $output_script:$output_amount)


### Sign the transaction

**Fetch previous output script**


In [39]:
# bx fetch-tx --format json [previous tx id] 

# previous_output_script=

previous_output_script=$(bx fetch-tx --format json $previous_txid \
| jq -r ".transaction.outputs[5].script")
echo $previous_output_script


connection timed out



**Sign transaction with previous output script.**

In [40]:
# bx input-sign [private key] "previous output script" [transaction template]

# signature= 

signature=$(bx input-sign --sign_type all --index 0 $privatekey_44h_1h_1h_0_5 "$previous_output_script" $csv_tx)


**Set the input script into the finalised transaction.**

In [ ]:
# bx input-set "[signature] [public key point]" [transaction template]

# my_tx=

my_tx=$(bx input-set --index 0 "[$signature] [$publickey_44h_1h_1h_0_5]" $my_tx)


In [41]:
# Validate.
bx validate-tx $my_tx


connection timed out


: 255

In [ ]:
# Decode
bx tx-decode -f json $my_tx |jq
